In [1]:
# The word vector is averaged directly to obtain the sentence vector

from gensim import utils
def read_corpus(fname):
    for line in open(fname,encoding = 'utf-8'):
        yield utils.simple_preprocess(line)

# 50000_WoS.txt
# 50000_MedLine.txt
train_corpus = list(read_corpus('../datasets/50000_MedLine.txt'))
len(train_corpus)

50000

In [2]:
from gensim.models import word2vec
import numpy as np 

model = word2vec.Word2Vec(sentences = train_corpus, vector_size=200, min_count = 5, epochs = 50) # Med:50 WOS:100
# default: 100, 5, 5
word_list = model.wv.index_to_key            #len(word_list) = 30064
vectors = np.asarray(model.wv.vectors)      #len(vectors) = 30064

In [3]:
# print(word_list[:10])
# print(len(word_list))
# print(model.wv['the'])

In [4]:
import numpy as np
from tqdm import tqdm
word_vec_per_doc = [[]*len(train_corpus) for x in range(len(train_corpus))]
doc_vector = [[]*len(train_corpus) for x in range(len(train_corpus))]
for docid in tqdm(range(len(train_corpus))):
    for wordid in range(len(train_corpus[docid])):  
        if (train_corpus[docid][wordid] in word_list):   # First determine if the term exists. Terms with too low frequency have been deleted
            word_vec_per_doc[docid].append(model.wv[train_corpus[docid][wordid]])
    doc_vector[docid] = np.array(word_vec_per_doc[docid]).mean(axis = 0)   # The word vector is averaged to obtain the sentence vector

100%|██████████| 50000/50000 [18:15<00:00, 45.66it/s]


In [5]:
from sklearn.cluster import KMeans
classNumber = 10
kmean_model = KMeans(n_clusters = classNumber).fit(doc_vector)
labels = kmean_model.labels_

from collections import Counter
center_dict = Counter(labels)
center_dict

Counter({1: 7194,
         5: 4867,
         3: 3813,
         0: 4821,
         4: 4178,
         8: 6208,
         2: 4176,
         6: 6234,
         7: 3426,
         9: 5083})

In [6]:
labels[:10]

array([1, 1, 1, 5, 1, 1, 1, 1, 1, 1])

In [7]:
def get_ground_truth_label():
    ground_truth_label = []
    # 50000_WoS_WC.txt
    # 50000_MedLine_Label.txt
    with open('../datasets/50000_MedLine_Label.txt','r',encoding = 'utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = int(line.replace('\n',''))
            ground_truth_label.append(line)
    return ground_truth_label
ground_truth_label = get_ground_truth_label()

In [8]:
from sklearn import metrics
print(metrics.adjusted_rand_score(labels, ground_truth_label))
print(metrics.fowlkes_mallows_score(labels, ground_truth_label))
print(metrics.adjusted_mutual_info_score(labels, ground_truth_label))

0.3235397106298138
0.39302785434854465
0.4405198271153591
